<a href="https://colab.research.google.com/github/khaledsoudy-1/gpt-3-5-turbo-finetuning-tutorial/blob/main/Fine_Tune_GPT_3_5_Turbo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤖 **Introduction to Fine-Tuning GPT-3.5 Turbo**

Fine-tuning is like giving a pre-trained model, such as **GPT-3.5** Turbo, a specialized "training session" to perform better on your specific tasks. Instead of starting from zero, you're building on a powerful model that already understands a lot about language, and guiding it to excel in your area of interest. 🎯

### **Why Fine-Tune GPT-3.5 Turbo?**

- **🔥 Boost Performance:** Fine-tuning can help the model perform much better on tasks that matter to you—whether it's answering questions, generating creative content, or even mastering a specific language (like Arabic!).
- **🎨 Task Specialization:** Want GPT-3.5 Turbo to get even better at something specific, like technical writing, summarizing articles, or even customer service? Fine-tuning is the key to unlocking that potential.
- **💡 Personalization:** Customize GPT-3.5 Turbo to match your tone, style, and preferences, making its responses feel more personal and on-point for your needs.

### **How Does Fine-Tuning Work?**

At a high level, fine-tuning is like showing GPT-3.5 Turbo a set of examples that guide it to respond better to your particular type of task. Think of it as teaching a model to become more "in tune" with your unique requirements. 🤖✨

Here’s a quick rundown of how the magic happens:

1. **Prepare Your Dataset 📂:** The first step is gathering a set of input-output examples that are specific to what you want the model to learn. These examples could be prompts (questions or tasks) and the desired responses (answers or completions).
   
2. **Upload Your Dataset 📤:** Once you've got your dataset, you’ll upload it to OpenAI’s API. The data should be in JSON format, with each entry containing a `prompt` and a `completion`—essentially, the question and the answer.

3. **Fine-Tune the Model 💪:** After uploading your dataset, you can kick off the fine-tuning process! You’ll specify the model (like `gpt-3.5-turbo`), point to your dataset, and configure the fine-tuning job.

4. **Evaluate the Model 🧐:** Once the fine-tuning is done, it’s time to see how the model performs! Test it on some new examples to make sure it’s giving you the responses you need.

5. **Refine & Repeat 🔄:** Don’t worry if the first result isn’t perfect—fine-tuning is an iterative process. Refine your dataset, tweak the parameters, and continue improving the model!

In the next sections, we'll dive deeper into each step, showing you exactly **how to fine-tune GPT-3.5-Turbo**, step by step. Ready to get started? Let’s go! 🚀


# 🚀 Step 1: Install the Required Library

First, we need to install the `openai` library to access OpenAI’s API.

In [1]:
! pip install openai

# 📲 Step 2: Connect to OpenAI’s API

After installing, import the `OpenAI` class and set up a client to interact with the API. You’ll need an API key from OpenAI.

In [7]:
from google.colab import userdata
api_key = userdata.get('OpenAI_API')

if api_key:
  print(f"OpenAI API Key: {api_key[:5]}...")

OpenAI API Key: sk-pr...


In [8]:
from openai import OpenAI

client = OpenAI(
    api_key=api_key
)

- **`userdata`:** This module from Google Colab allows us to securely access secrets stored within Colab's environment.
- **`api_key`**: Your private key to access the OpenAI API.
- **`OpenAI`**: The main class to interact with OpenAI’s services.
- The **`api_key`** is passed to **`OpenAI`** so it can authenticate with the **OpenAI API** for making requests, such as starting a fine-tuning job or generating text.

> ⚠️ **Tip**: Never share your API key publicly to keep your account secure!
>

# 📂 Step 3: Load Data from Google Drive

Since Colab sessions reset, storing data on Google Drive makes it easily accessible. Mount Google Drive so Colab can interact with it.

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 🔗 Step 4: Set File Paths

Specify paths to your training and validation files in JSONL format. These files contain examples to help the model learn during fine-tuning.

In [10]:
training_file = "/content/drive/MyDrive/Colab Notebooks/04- Fine-Tune GPT 3.5 Turbo /training_data.jsonl"

validation_file = "/content/drive/MyDrive/Colab Notebooks/04- Fine-Tune GPT 3.5 Turbo /validation_data.jsonl"

# 📄 Step 5: Upload Files to OpenAI

Upload the files to OpenAI’s servers, setting the purpose to `"fine-tune"` so they know it’s for training.

In [50]:
training_file_obj = client.files.create(
  file=open(training_file, "rb"),
  purpose="fine-tune"
)


validation_file_obj = client.files.create(
  file=open(validation_file, "rb"),
  purpose="fine-tune"
)

print(f"Training File Object: {training_file_obj}\n")
print(f"validation File Object: {validation_file_obj}")

Training File Object: FileObject(id='file-tkKgy3dx3YbXvm16SUEGRYgT', bytes=5181, created_at=1731503978, filename='training_data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

validation File Object: FileObject(id='file-g7CWUEgXa7PDgBvxZPUbZxc1', bytes=2685, created_at=1731503978, filename='validation_data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)


- **`client.files.create()`**: This method **uploads a file** to OpenAI’s server.
- **`open(file_name, "rb")`** opens the data file (stored in the variable **training_file** or **validation_file**) in **binary read mode**.
- **`file=`** parameter is where you specify the file to upload.
- **`purpose="fine-tune"`**: This argument tells OpenAI's API that this file is meant specifically for fine-tuning, so the API processes it accordingly.

In [51]:
training_file_id = training_file_obj.id

validation_file_id = validation_file_obj.id

print(f"Training File ID: {training_file_id}\n")
print(f"validation File ID: {validation_file_id}")

Training File ID: file-tkKgy3dx3YbXvm16SUEGRYgT

validation File ID: file-g7CWUEgXa7PDgBvxZPUbZxc1


# 🔧 Step 6: Start the Fine-Tuning Job

Now that we’ve uploaded the files, we can start the fine-tuning job by providing the `training_file_id`, `validation_file_id`, and model name (`"gpt-3.5-turbo"`).

In [52]:
fine_tuning_job_response = client.fine_tuning.jobs.create(
  training_file=training_file_id,
  validation_file=validation_file_id,
  model="gpt-3.5-turbo",
  hyperparameters={
      "n_epochs": 15,
      "batch_size": 3,
      "learning_rate_multiplier": 0.3,
  },
)

fine_tuning_job_response

FineTuningJob(id='ftjob-XdWROGbBqI1j9vRFjDdX3jst', created_at=1731503980, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=15, batch_size=3, learning_rate_multiplier=0.3), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-mgfjI7HyLPZIak99Sy4VvIW5', result_files=[], seed=2061283848, status='validating_files', trained_tokens=None, training_file='file-tkKgy3dx3YbXvm16SUEGRYgT', validation_file='file-g7CWUEgXa7PDgBvxZPUbZxc1', estimated_finish=None, integrations=[], user_provided_suffix=None)

- **`client.fine_tuning.jobs.create(...)`**: This initiates a fine-tuning job using OpenAI’s API.
- **`hyperparameters={...}`**: A dictionary specifying training settings for fine-tuning:
- **`n_epochs`**: Sets the number of times the model will iterate over the training data.
- **`batch_size`**: Number of examples processed at once (higher numbers can be faster but require more memory).
- **`learning_rate_multiplier`**: Adjusts the learning rate to control how quickly the model learns.

In [53]:
# Get the Job ID
job_id = fine_tuning_job_response.id

# Get the Job Status
job_status = fine_tuning_job_response.status

print(f"Fine Tuning Job ID: {job_id}\n")
print(f"Fine Tuning Job Status: {job_status}")

Fine Tuning Job ID: ftjob-XdWROGbBqI1j9vRFjDdX3jst

Fine Tuning Job Status: validating_files


### ❗❗ This is step is **crusial** If Google Colab's Run-Time shuts off.

In [22]:
jobs = client.fine_tuning.jobs.list()  # List all fine-tuning jobs

for job in jobs.data:
    print(f"Job ID: {job.id} - Status: {job.status}")

Job ID: ftjob-XdWROGbBqI1j9vRFjDdX3jst - Status: succeeded
Job ID: ftjob-HOzhngnH6aESL3yzmLq6vD98 - Status: succeeded
Job ID: ftjob-EEzs9MqqAy53nnpQ5eJt6Hdi - Status: succeeded
Job ID: ftjob-tYQ8qGwhoAvSN4mfK8mCDehn - Status: succeeded
Job ID: ftjob-FnSdcTx98w5ypCKS1EsiG1IT - Status: succeeded


# ⏳ Step 7: Monitor Training Progress

Using Python’s `signal` module, set up a handler to monitor the events of a fine-tuning job and gracefully handle interruptions, such as the user stopping the script (by pressing **Ctrl+C**)..

In [16]:
import signal
import datetime

job_id = "ftjob-XdWROGbBqI1j9vRFjDdX3jst"       # In Case Google Colab run-time goes off

# Create a function for Handling interruptions (e.g., Ctrl+C)
def signal_handler(sig, frame):
  job_status = client.fine_tuning.jobs.retrieve(fine_tuning_job_id=job_id).status          # Status of the job ('succeeded' or 'failed')
  print(f"Stream interrupted. The fine-tuning job status is {job_status}")
  return

# Set up signal handler for interruption
signal.signal(signal.SIGINT, signal_handler)

print(f"Streaming events for the fine-tuning job: {job_id}\n")

# Fetch and display events for the fine-tuning job
try:
  events = client.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id)

  for event in events:
    event_time = datetime.datetime.fromtimestamp(event.created_at)
    event_message = event.message

    print(f"{event_time} - {event_message}")       # Print time and message for each event

except Exception as e:
  print(f"Stream Interrupted (Client Disconnected): {e}")

Streaming events for the fine-tuning job: ftjob-XdWROGbBqI1j9vRFjDdX3jst

2024-11-13 13:24:38 - The job has successfully completed
2024-11-13 13:24:35 - New fine-tuned model created
2024-11-13 13:24:35 - Checkpoint created at step 54
2024-11-13 13:24:35 - Checkpoint created at step 51
2024-11-13 13:24:21 - Step 55/55: training loss=0.63, validation loss=0.59
2024-11-13 13:24:21 - Step 54/55: training loss=0.67, validation loss=0.58, full validation loss=0.60
2024-11-13 13:24:16 - Step 53/55: training loss=0.90, validation loss=0.68
2024-11-13 13:24:14 - Step 52/55: training loss=0.60, validation loss=0.56
2024-11-13 13:24:14 - Step 51/55: training loss=0.73, validation loss=0.70, full validation loss=0.60
2024-11-13 13:24:09 - Step 50/55: training loss=0.80, validation loss=0.56
2024-11-13 13:24:07 - Step 49/55: training loss=0.61, validation loss=0.74
2024-11-13 13:24:07 - Step 48/55: training loss=0.63, validation loss=0.59, full validation loss=0.60
2024-11-13 13:24:02 - Step 47/55:

- **`signal`:** for handling interrupts (like Ctrl+C).
- `datetime`: for converting timestamps to readable dates
- **`signal_handler(sig, frame)`**: Defines a function to handle signals like **`SIGINT`** (**Ctrl+C**). If the script is interrupted, it retrieves the current job status and prints it, letting the user know the job’s state.
- **`sig`** and **`frame`** are **parameters of the signal handler function**, and the **`signal`** module passes them automatically when a signal (like **`SIGINT`**) is received.
- **`signal.signal(signal.SIGINT, signal_handler)`**: Tells Python to call **`signal_handler`** when a **`SIGINT`** (interrupt) signal is received, helping manage unexpected terminations gracefully.
- **`client.fine_tuning.jobs.list_events(job_id)`**: Retrieves a list of events related to the fine-tuning job identified by **`job_id`**.

# ⏱ Step 8: Wait for Completion

Now, check if the job has finished by periodically retrieving its **status**.

In [17]:
import time

job_id = "ftjob-XdWROGbBqI1j9vRFjDdX3jst"                 # In Case Google Colab run-time goes off

# Retrieve the status of the fine-tuning job
job_status = client.fine_tuning.jobs.retrieve(fine_tuning_job_id=job_id).status

if job_status not in ['succeeded', 'failed']:
  print(f"Job is still running. Current status: {job_status}. Waiting...")

  while job_status not in ['succeeded', 'failed']:
    time.sleep(2)
    job_status = client.fine_tuning.jobs.retrieve(fine_tuning_job_id=job_id).status          # Check status again
    print(f"Job Status: {job_status}")

else:
  print(f"Fine-tune Job {job_id} finished with status {job_status}")

Fine-tune Job ftjob-XdWROGbBqI1j9vRFjDdX3jst finished with status succeeded


- **`client.fine_tuning.jobs.retrieve(job_id).status`**: This retrieves the status of the fine-tuning job using the **`job_id`**. It makes an API call to get information about the fine-tuning job, and the **`.status`** accesses the status of the job (e.g., ”**pending**”, “**in_progress**”, “**succeeded**”, “**failed**”).
- **`if status not in ["succeeded", "failed"]:`**: This checks if the current status of the job is not one of the terminal statuses (**Succeeded** or **Failed**). Terminal statuses mean that the job has completed (either successfully or with failure).
- **`while status not in ["succeeded", "failed"]:`**: This loop will keep running as long as the job status is neither **Succeeded** nor **Failed**. It continuously checks the job status.
- **`time.sleep(2)`**: This pauses the execution of the code for 2 seconds before checking the status again. This helps avoid overwhelming the API with continuous requests.
- **`status = client.fine_tuning.jobs.retrieve(job_id).status`**: After the 2-second pause, the status of the job is checked again.
- **`else:`**: If the job has finished (i.e., it’s either  **Succeeded** or **Failed**), the code will break out of the **`while`** loop and print the final status of the fine-tuning job.

# 🏆 Step 9: Check Fine-Tuning Results

When fine-tuning completes, view all jobs and find the **ID** of the **Fine-Tuned Model**.

In [18]:
# Retrieve a list of all fine-tuning jobs from the OpenAI API

result = client.fine_tuning.jobs.list()
result.data

[FineTuningJob(id='ftjob-XdWROGbBqI1j9vRFjDdX3jst', created_at=1731503980, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal::AT7eFB0B', finished_at=1731504273, hyperparameters=Hyperparameters(n_epochs=15, batch_size=3, learning_rate_multiplier=0.3), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-mgfjI7HyLPZIak99Sy4VvIW5', result_files=['file-TCjIropahBX9R4BwZaqFjkwD'], seed=2061283848, status='succeeded', trained_tokens=25140, training_file='file-tkKgy3dx3YbXvm16SUEGRYgT', validation_file='file-g7CWUEgXa7PDgBvxZPUbZxc1', estimated_finish=None, integrations=[], user_provided_suffix=None),
 FineTuningJob(id='ftjob-HOzhngnH6aESL3yzmLq6vD98', created_at=1731497755, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal::AT61Peal', finished_at=1731498021, hyperparameters=Hyperparameters(n_epochs=15, batch_size=3, learning_rate_multiplier=0.3), model='gpt-3.5-turbo-0125',

- **`client.fine_tuning.jobs.list()`** sends a request to the **OpenAI API** to fetch all the fine-tuning jobs that have been performed or are in progress. It returns a response object that contains various details about the fine-tuning jobs.
- **`result`** : It is an object that contains a **`.data`** attribute that holds the list of fine-tuning jobs.


In [19]:
# Print the total number of fine-tuning jobs retrieved

print(f"Found {len(result.data)} Fine-Tuning Jobs.")

Found 5 Fine-Tuning Jobs.


In [20]:
# Extract the fine-tuned model ID from the first fine-tuning job in the list
fine_tuned_model_id = result.data[0].fine_tuned_model

# Print the Fine-Tuned Model ID
print(f"Fine-Tuned Model ID: {fine_tuned_model_id}")

Fine-Tuned Model ID: ft:gpt-3.5-turbo-0125:personal::AT7eFB0B


- **`result.data[0]`**: This accesses the first fine-tuning job in the list of jobs (**`data`** is the list holding the jobs).
- **`.fine_tuned_model`**: This accesses the specific attribute of that job which holds the information about the **fine-tuned model**.
- `print(fine_tuned_model_id)`: prints the **ID** or details of the **fine-tuned model** that was extracted in the previous step.
- **`ft:`** to indicate that it’s a fine-tuned model

# 🗣 Step 10: Test the Fine-Tuned Model

Now, test your fine-tuned model by sending it a prompt. **Compare** its response to the base GPT-3.5 model.

## 1. Base Model Response

In [45]:
# Send a prompt to the base (non-fine-tuned) model and get the response

base_model_response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "إنت مساعد شخصي بتجاوب العملاء على استفساراتهم"},
        {"role": "user", "content": "إيه سياسة إرجاع الكتب عندكم؟"}
        ]
)

print("Base Model Response:")
print(base_model_response.choices[0].message.content)

Base Model Response:
سياسة إرجاع الكتب لدى معظم المكتبات تتباين قليلاً من مكتبة إلى أخرى. ولكن بشكل عام، يمكن إرجاع الكتب في معظم المكتبات خلال فترة زمنية معينة بعد شرائها مقابل استرداد قيمتها أو استبدالها بكتب أخرى. يمكن أن تكون هناك شروط معينة لعملية الإرجاع مثل حالة الكتاب وتواجده في حالة جيدة، وعادةً يجب أن يكون الإيصال الأصلي متوفرًا. لذا، يفضل التحقق من سياسة الإرجاع لدى المكتبة المحددة التي تتسوق منها للحصول على تفاصيل دقيقة حول كيفية إرجاع الكتب.


## 2. Fine-Tuned Model Response

In [46]:
# Send the same prompt to the fine-tuned model and get the response

fine_tuned_model_response = client.chat.completions.create(
    model=fine_tuned_model_id,                # Fine-Tuned Model ID
    messages=[
        {"role": "system", "content": "إنت مساعد شخصي بتجاوب العملاء على استفساراتهم"},
        {"role": "user", "content": "إيه سياسة إرجاع الكتب عندكم؟"}
        ]
)

print("Fine-Tuned Model Response:")
print(fine_tuned_model_response.choices[0].message.content)

Fine-Tuned Model Response:
مرحباً! سياسة إرجاع الكتب عندنا تتضمن مدة إرجاع تبلغ 14 يوماً من تاريخ الشراء، على أن تكون الكتب بحالتها الأصلية وغير مستخدمة. بما أني مساعد افتراضي، قد تحتاج تأكيد هذه المعلومات من مكان الشراء، فتأكد من مراجعة سياسة الإرجاع على الموقع الإلكتروني الخاص بالمتجر. تحتاج مساعدة بشيء آخر؟
